In [ ]:
import pandas as pd
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import os
import chromedriver_autoinstaller
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")
opt.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])
# opt.add_argument("--headless")
chromedriver_autoinstaller.install()

In [ ]:
CREATE_CONFIG_FLAG = False
SAVE_DATA_FLAG = True
UPDATE_CONFIG_FLAG=True
# value in config file status column to trigger scrapper
STATUS_VALUE_TO_SCRAPE=['No','Fail']

In [ ]:
def scroll_down_page(driver, speed=8,sleep_interval=1):
    current_scroll_position, new_height= 0, 1
    while current_scroll_position <= new_height:
        current_scroll_position += speed
        driver.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
        new_height = driver.execute_script("return document.body.scrollHeight")
        sleep(1)

In [ ]:
def scroll_down_slowly(driver, speed=3000, sleep_interval=3,scrolling_steps=20):
    y = speed
    for timer in range(0,scrolling_steps):
        driver.execute_script("window.scrollTo(0, "+str(y)+")")
        y += speed  
        sleep(sleep_interval)

In [ ]:
def load_dynamic_main_table(
    web_driver,
    url,
    sleep_interval,
    speed=3000,
    scroll_sleep_interval=3,
    scrolling_steps_count=20    
    ):
    web_driver = web_driver
    url = url
    web_driver.get(url)
    sleep(sleep_interval)
    print("scrolling to laod complete main table")
    scroll_down_slowly(web_driver,speed,scroll_sleep_interval,scrolling_steps_count)
    return True


In [ ]:
def load_dynamic_main_table_old(
    web_driver,
    url,
    range_input=1000,
    sleep_interval=5,
    speed=300,
    scroll_sleep_interval=3
    ):
    web_driver = web_driver
    url = url
    web_driver.get(url)
    sleep(sleep_interval)
    print("scrolling to laod complete main table")
    previous_length = 0
    counter =0
    for x in range (range_input):
        print("scrolling down to end page")
        # web_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # scroll_down_page(web_driver,speed,scroll_sleep_interval)
        scroll_down_slowly(web_driver,speed,scroll_sleep_interval)
        print("sleeping")
        sleep(1)
        print("woke")
        results=web_driver.find_elements(by=By.XPATH, value='.//table[@class="table box table-striped"]')
        if previous_length <= len(results):
            counter += 1
        else:
            counter = 0
        previous_length = len(results)
        table_html = results[-1].get_attribute("outerHTML")
        # check if table is empty
        df = pd.read_html(table_html)[0]
        if df.shape[0] == 0:
            break
        if counter > 20:
            print("table loading finished")
            break
    return True

In [ ]:
def fetch_main_table_from_loaded_page(driver):
    final_dfs=[]
    print("retrieving all data.... \n")

    #     get all table's data and concatenate them
    results=driver.find_elements(by=By.XPATH, value='.//table[@class="table box table-striped"]')
    for each_table in results:
        # Use Pandas to read HTML and convert it to a DataFrame
        df = pd.read_html(each_table.get_attribute("outerHTML"))[0]
        links = []
        for row_index, row in df.iterrows():
            link_elements = each_table.find_elements(by=By.XPATH, value=f'.//tr[{row_index + 1}]/td[1]//a')
            # Extract the href attribute from the first link element, if any
            link = link_elements[0].get_attribute("href") if link_elements else None
            links.append(link)
        # Add a new column to the DataFrame with the extracted links
        df["Link"] = links
        final_dfs.append(df)
    result_df=pd.concat(final_dfs, axis=0, ignore_index=True)
    return result_df

### Main

In [ ]:
data_folder="../data/extra/clubranking/insidelacrosse"
if (not os.path.exists(data_folder)):
    os.makedirs(data_folder)
    print(f'new data folder {data_folder} created success')

In [ ]:
club_ranking_main_url = "https://www.insidelacrosse.com/recruiting/club"
gender_list=["girls","boys"]
class_list = [x for x in range(2024,2029)]
session_list = [x for x in range(2019,2025)]

In [ ]:
# create config file to track status and resume scraping
if CREATE_CONFIG_FLAG:

    urls = []
    status=[]
    for gender in gender_list:
        for clas in class_list:
            for session in session_list:
                url_to_scrape = club_ranking_main_url+f'/{clas}/{session}/{gender}'
                urls.append(url_to_scrape)
                status.append('No')
    scraping_status_df=pd.DataFrame({'url':urls,'status':status})
    scraping_status_df.to_csv(os.path.join(data_folder,'input','url_scraping_status.csv'),index=False)
    scraping_status_df.head()

In [ ]:
# read scraping status csv file
scraping_status_df = pd.read_csv(os.path.join(data_folder,'input','url_scraping_status.csv'))
# scraping_status_df.head()

In [ ]:
# initiate webdriver
driver = webdriver.Chrome(options=opt)

In [ ]:
# iterate over dataframe
for index, row in scraping_status_df.iterrows():
    if row['status'] in STATUS_VALUE_TO_SCRAPE:
        # start scraping data
        try:
            url_to_scrape = row['url']
            print(url_to_scrape)
            load_dynamic_main_table(driver,row['url'],sleep_interval=5,speed=2000,scroll_sleep_interval=3)
            output_df=fetch_main_table_from_loaded_page(driver)
            file_to_save = url_to_scrape.split('club/')[-1].replace('/','_')+'.csv'
            print(file_to_save)
            if SAVE_DATA_FLAG:
                output_df.to_csv(os.path.join(data_folder,'output/raw',file_to_save),index=False)
            print('saved success')
            scraping_status_df.loc[index,'status']='Yes'
            del(output_df)
        except Exception as e:
            print(e)
            print('data save failed')
            scraping_status_df.loc[index,'status']='Fail'
            # update status col with Failed
            # scraping_status_df
        if index > 2 and index%2 ==0:
            if UPDATE_CONFIG_FLAG:
                scraping_status_df.to_csv(os.path.join(data_folder,'input','url_scraping_status.csv'),index=False)
            print(f'config file updated')
    else:
        # url =  row['url']
        # status = row['status']
        print(f"skipping url: {row['url']} with status: {row['status']}")
    # if index >= 5:
    #     break

In [ ]:
driver.close()